In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time, sys

class spotWrapper():
    def __init__(self):
        client_id = 'accd0aa479164ddcbf1cbf822512b80b'
        client_secret = '58bfc467435045e7b61c86fb03385729'
        client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
        self.spot = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    def getAlbumsTracks(self, id):
        return self.spot.album_tracks(id)
    def getAudioFeatures(self, song_id):
        return self.spot.audio_features(song_id)



In [2]:
spot = spotWrapper()
artist_df = pd.read_excel("SpotifyArtists.xlsx")
all_artists = artist_df[["spotify_name", "artist_uri"]]

album_df = pd.read_excel("SpotifyAlbums.xlsx")
all_albums = album_df["album_uri"]

print("getting songs for: " + str(len(all_albums)) + " albums.")
song_uri, song_name, album_uri, artist_uris, artist_names= ([] for i in range(5))
columns = ["song_uri", "song_name", "album_uri", "artist_uris", "artist_names", "explicit", "preview_url"]
counter = 0
for idx in all_albums:
        # search on album URI   
    #print("Album number "+ str(counter) )
    counter+=1
    if(counter > 30):
        break
    album_data = spot.getAlbumsTracks(idx)["items"]
    temp_artists, temp_artist_uris = ([] for x in range(2))
    number_of_artists = len(album_data[0]["artists"])
    for num in range(0,number_of_artists):
        artist_name_in_list = album_data[0]["artists"][num]["name"]
        try:
            artist_uri_in_list = all_artists.loc[all_artists["spotify_name"] == artist_name_in_list, "artist_uri"].iloc[0]
        except:
            artist_uri_in_list = "0"
        temp_artists.append(artist_name_in_list)
        temp_artist_uris.append(artist_uri_in_list)
    album_artists = ", ".join(temp_artists)
    album_artist_uris = ", ".join(temp_artist_uris)

        # populate lists with song data
    for song_num in range(0,len(album_data)):  
        album_uri.append(idx)  
        artist_uris.append(album_artist_uris)
        artist_names.append(album_artists)
        song_uri.append(album_data[song_num]["uri"])
        name = album_data[song_num]["name"]
        song_name.append(name)

    # concatenate the lists 
song_df = pd.concat([pd.Series(song_uri), pd.Series(song_name), pd.Series(album_uri),pd.Series(artist_uris),
                                        pd.Series(artist_names)], 
                                        axis=1, keys=columns)
#print(song_df)
dance, energy, loudness, valence, tempo, instru, speech = ([] for i in range(7))
for i in range(0,len(song_df)):
    #print(i)
    uri = song_df["song_uri"][i]
    spot_audio = spot.getAudioFeatures(uri)

        # add song data to lists of metrics
    dance.append(spot_audio[0]['danceability'])
    energy.append(spot_audio[0]['energy'])
    loudness.append(spot_audio[0]['loudness'])
    valence.append(spot_audio[0]['valence'])
    tempo.append(spot_audio[0]['tempo'])
    instru.append(spot_audio[0]['instrumentalness'])
    speech.append(spot_audio[0]['speechiness'])
interm = pd.concat([song_df, pd.Series(dance), pd.Series(energy), pd.Series(loudness),pd.Series(valence),
                                        pd.Series(tempo), pd.Series(instru), pd.Series(speech)], 
                                        axis=1)


getting songs for: 508 albums.


Max Retries reached

KeyboardInterrupt



In [10]:

def get_song_metrics(df):
    spot = spotWrapper()
    dance, energy, loudness, valence, tempo, instru, speech = ([] for i in range(7))
    all_uris = df["song_uri"]
    length = len(all_uris)
    for i in range(0,length-6700):
        print(i)
        time.sleep(0.5)
        if(i%50 == 0):
            time.sleep(0.5)
            print("Song # "+str(i)+" of "+ str(length ))
        uri = all_uris[i]
        print(uri)
        spot_audio = spot.getAudioFeatures(uri)
        print(spot_audio)
        #if(spot_audio == None):
            #print("problem detected: " + uri)
            #dance.append(0)
            #energy.append(0)
     #       loudness.append(0)
      #      valence.append(0)
       #     tempo.append(0)
        #    instru.append(0)
        #    speech.append(0)
    #    else:
        # add song data to lists of metrics
     #       dance.append(spot_audio[0]['danceability'])
      #      energy.append(spot_audio[0]['energy'])
       #     loudness.append(spot_audio[0]['loudness'])
        #    valence.append(spot_audio[0]['valence'])
         #   tempo.append(spot_audio[0]['tempo'])
          #  instru.append(spot_audio[0]['instrumentalness'])
           # speech.append(spot_audio[0]['speechiness'])
    #return pd.concat([df, pd.Series(dance), pd.Series(energy), pd.Series(loudness),pd.Series(valence),
     #                                   pd.Series(tempo), pd.Series(instru), pd.Series(speech)], 
      #                                  axis=1)




In [11]:
main = pd.read_excel("SpotifySongs.xlsx")
with_metrics = get_song_metrics(main)
print(with_metrics)

0
Song # 0 of 8628
spotify:track:57q6BgYP4gN6hPQViCB3um


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=57q6BgYP4gN6hPQViCB3um:
 Max Retries, reason: too many 429 error responses

In [5]:
main["song_uri"]

0       spotify:track:57q6BgYP4gN6hPQViCB3um
1       spotify:track:3oHkMCVJyOcjg5FhfLc2Rq
2       spotify:track:52eYVUkFTOVozbVFIaFrnV
3       spotify:track:1AqemxWsT0iGg5Lvkm16Bt
4       spotify:track:7ycWLEP1GsNjVvcjawXz3z
                        ...                 
8623    spotify:track:1u3hwQVMWxZUVR56sZg1Sk
8624    spotify:track:4HppSdg3ZOAyQR9ZKqNNwg
8625    spotify:track:4zPrRIdH6oN7eArKvo8iV0
8626    spotify:track:0LBGINk4SrnjYKPCuQu3jU
8627    spotify:track:56KPYkBUA0GWxUQwMTJlNU
Name: song_uri, Length: 8628, dtype: object